In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LSTM, Bidirectional
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [3]:
data = pd.read_csv('/content/drive/MyDrive/fer2013/icml_face_data.csv')
data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
# Split the data into training, validation, and testing sets
train_data = data[data['Usage'] == 'Training']
val_data = data[data['Usage'] == 'PublicTest']
test_data = data[data['Usage'] == 'PrivateTest']

In [5]:
# Convert the pixels from strings to arrays of integers
X_train = np.array([np.fromstring(x, dtype=int, sep=' ') for x in train_data['pixels']])
X_val = np.array([np.fromstring(x, dtype=int, sep=' ') for x in val_data['pixels']])
X_test = np.array([np.fromstring(x, dtype=int, sep=' ') for x in test_data['pixels']])

In [6]:
# Reshape the data to be 48x48 grayscale images
X_train = X_train.reshape(-1, 48, 48, 1)
X_val = X_val.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

In [7]:
# Convert the labels to one-hot encoding
y_train = np_utils.to_categorical(train_data['emotion'], 7)
y_val = np_utils.to_categorical(val_data['emotion'], 7)
y_test = np_utils.to_categorical(test_data['emotion'], 7)

In [8]:
model = Sequential()

# First LSTM layer
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(48, 48)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Second LSTM layer
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Output layer
model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 48, 256)          181248    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 48, 256)           0         
                                                                 
 batch_normalization (BatchN  (None, 48, 256)          1024      
 ormalization)                                                   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1, validation_data=(X_val, y_val), callbacks=callbacks_list)

Epoch 1/50
449/449 [==============================] - ETA: 0s - loss: 2.0518 - accuracy: 0.2107
Epoch 1: val_accuracy improved from -inf to 0.28392, saving model to model.h5
449/449 [==============================] - 193s 406ms/step - loss: 2.0518 - accuracy: 0.2107 - val_loss: 1.7621 - val_accuracy: 0.2839
Epoch 2/50
449/449 [==============================] - ETA: 0s - loss: 1.7970 - accuracy: 0.2600
Epoch 2: val_accuracy did not improve from 0.28392
449/449 [==============================] - 180s 402ms/step - loss: 1.7970 - accuracy: 0.2600 - val_loss: 2.0706 - val_accuracy: 0.2065
Epoch 3/50
449/449 [==============================] - ETA: 0s - loss: 1.7654 - accuracy: 0.2780
Epoch 3: val_accuracy improved from 0.28392 to 0.29423, saving model to model.h5
449/449 [==============================] - 183s 407ms/step - loss: 1.7654 - accuracy: 0.2780 - val_loss: 1.7379 - val_accuracy: 0.2942
Epoch 4/50
449/449 [==============================] - ETA: 0s - loss: 1.7483 - accuracy: 0.2876
E

In [ ]:
from keras.models import load_model

model = load_model('model.h5')

In [ ]:
new_data = np.array([np.fromstring(x, dtype=int, sep=' ') for x in new_data_pixels])
new_data = new_data.reshape(-1, 48, 48, 1)
predictions = model.predict(new_data)